In [ ]:
from pystac import Item
from odc.stac import load, configure_s3_access
from ipyleaflet import basemaps
from odc.geo.geom import BoundingBox

In [ ]:
configure_s3_access(aws_unsigned=True)

In [ ]:
row, col = (108, 8)
# row, col = (51, 30)
year = 2017

base = (
    "https://dep-public-staging.s3.us-west-2.amazonaws.com/dep_s2_seagrass/0-2-0/"
    f"{row:03}/{col:03}/{year}/dep_s2_seagrass_{row:03}_{col:03}_{year}.stac-item.json"
)

items = [Item.from_file(base.format(year=year, row=row, col=col))]
data = load(items, fail_on_error=False)

data

In [ ]:
data.seagrass_probability.odc.explore()

In [ ]:
data.seagrass.where(data.seagrass == 1).plot.imshow(col="time", col_wrap=2, cmap="Greens_r", size=6)